In [ ]:
# TO USE, upload an api key from Kaggle (kaggle.json)
# Runtime -> Change runtime type -> Hardware accelerator set to GPU (optional)
# make a shortcut of the trained model into local gdrive and change following as required
model_path = "/content/gdrive/MyDrive/trained_bird_CNN"

In [ ]:
!pip install tensorflow
!pip install keras
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras
import numpy as np
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
model = keras.models.load_model(model_path)

In [ ]:
!mkdir -p  ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d gpiosenka/100-bird-species

import zipfile
local_zip = '/content/100-bird-species.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/')
zip_ref.close()

In [ ]:
test_preprocessor = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_preprocessor.flow_from_directory('/content/test',
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical')

In [ ]:
model.evaluate(test_generator,
               use_multiprocessing=True,
               workers=10)

In [ ]:
# https://keras.io/api/preprocessing/image/

def classify(img_path, model, label_map, top_n=3):
  img = image.load_img(img_path, grayscale=False, target_size=(224, 224))
  input_arr = image.img_to_array(img)
  input_arr = np.expand_dims(input_arr, axis=0)
  input_arr = preprocess_input(input_arr) # preprocess function specific to ResNet50

  predictions = model.predict(input_arr)

  top_n_indices = np.argpartition(predictions[0], -top_n)[-top_n:] # gets top_n prediction indices
  top_n_indices = top_n_indices[np.argsort(predictions[0][top_n_indices])] # sorts by prediction confidence
  top_n_indices = top_n_indices[::-1] # reverses array so that sorted most to least confident


  for index in top_n_indices:
    print(label_map[index], predictions[0][index])
  plt.matshow(img)

In [ ]:
label_map = (test_generator.class_indices) # maps index in predict array to species name
label_map = {v: k for k, v in label_map.items()} # inverts key-value pairs so that index is key

# for human evaluation of model performance on test images

species_name = "SCARLET MACAW"
import os
dir = "/content/test/" + species_name
for filename in os.listdir(dir):
  if filename.endswith("jpg"): 
    img_path = dir + "/" + filename
    print("\n" + img_path)
    classify(img_path, model, label_map, top_n=3)

In [ ]:
classify("/content/african firefinch.jpeg", model, label_map, top_n=3)